In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

df = pd.read_csv("actual_final_data.csv")
del df['Unnamed: 0']
df

,movieId,count,avg_rating,director,genre,year,LeadActor,language
0,131072.0,1.0,2.500000,Florian David Fitz,Comedy,2012,Jessica Schwarz,German
1,1.0,49695.0,3.921240,John Lasseter,Animation,1995,Tom Hanks,English
2,2.0,22243.0,3.211977,Joe Johnston,Adventure,1995,Robin Williams,English
3,3.0,12735.0,3.151040,Howard Deutch,Comedy,1995,Walter Matthau,English
4,4.0,2756.0,2.861393,Forest Whitaker,Comedy,1995,Whitney Houston,English
...,...,...,...,...,...,...,...,...
25986,131060.0,1.0,1.500000,Tobi Baumann,Comedy,2007,Oliver Pocher,German
25987,131062.0,1.0,0.500000,Jim Torres,Drama,2008,Joshua Leonard,English
25988,131066.0,1.0,3.000000,Kresten Vestbjerg Andersen,Animation,2011,Anders Juul,Danish
25989,131068.0,1.0,3.000000,Florian Gärtner,Comedy,2005,André Kaminski,German


# WHAT MUST BE DONE
1) create X (all data except movieId and rating) and Y (rating only) dataframes <br>
2) map Y values onto some finite set, e.g. 0-0.5 => 0, 0.5-1 => 1, etc. ('outputs' for neural net) <br>
3) encode X data (specifically director, LeadActor, genre, language) <br>
4) split into testing and training and do np.ravel or something on Y data so it is useable <br>
5) fit neural net to create models and make predictions, check accuracy <br>
6) check accuracy or whatever

In [ ]:
#Can't do pd.get_dummies because the dataframe is too large
#X = pd.get_dummies(df, columns=['director','genre','LeadActor','language'])

"""
del X['movieId']
del X['avg_rating']
#del X['count'] <--> NOT deleting because amount of times movies are rated has significance
X
"""

In [ ]:
"""
Y = df[['avg_rating']]
Y
"""

In [ ]:
#mapping different ratings to create neural net
#now the net will predict which range the movie rating will fall in
"""
for i in range(0, len(Y)):
    if Y.loc[i, 'avg_rating'] <= 1.0:
        Y.loc[i, 'avg_rating'] = np.uint8(0)
    elif Y.loc[i, 'avg_rating'] <= 2.0:
        Y.loc[i, 'avg_rating'] = np.uint8(1)
    elif Y.loc[i, 'avg_rating'] <= 3.0:
        Y.loc[i, 'avg_rating'] = np.uint8(2)
    elif Y.loc[i, 'avg_rating'] <= 4.0:
        Y.loc[i, 'avg_rating'] = np.uint8(3)
    else:
        Y.loc[i, 'avg_rating'] = np.uint8(4)
Y
"""

In [ ]:
#split data into training and testing
#+ then alter the Y data so it can be used correctly
"""
x_train, x_test, y_train_intermediate, y_test_intermediate = train_test_split(X, Y, test_size=0.3, random_state=35)
y_train = np.ravel(y_train_intermediate)
y_test = np.ravel(y_test_intermediate)
"""

In [ ]:
#create the classifier and fit it onto the training data
"""
clf = MLPClassifier(hidden_layer_sizes=(250,150), random_state=35, activation='logistic')
clf.fit(x_train, y_train)
"""

In [ ]:
#make some predictions and test the accuracy
"""
from sklearn.metrics import accuracy_score

y_train_pred = clf1.predict(x_train)
y_test_pred = clf1.predict(x_test)

print(accuracy_score(y_test, y_test_pred))
"""